In [1]:
import pandas as pd
import numpy as np
import itertools
import re
import unicodedata
import networkx as nx
from collections import defaultdict

In [2]:
df = pd.read_csv(r"D:\MS\Indiana University Bloomington\Classes\ENGR-E 583 Information Visualization (Dr. Katy Börner & Michael Ginda)\Client Project\Week 5\final_preprocessed_dataset.csv")

In [3]:
# --- STEP 1: Preprocess columns ---
df['Venue_Year'] = df['Venue'] + "_" + df['Year Started'].astype(str)
df['Start_Date'] = pd.to_datetime(df['Start Date']).dt.strftime('%Y-%m-%d')
df['End_Date'] = pd.to_datetime(df['End Date']).dt.strftime('%Y-%m-%d')

def split_organizers(org):
    if pd.isna(org):
        return []
    return [o.strip() for o in re.split(r',|/|&|;', str(org)) if o.strip()]

df['Organizer_List'] = df['Organizer'].apply(split_organizers)

# --- STEP 2: Build edges based on shared organizers ---
organizer_to_venues = defaultdict(set)
for _, row in df.iterrows():
    for org in row['Organizer_List']:
        organizer_to_venues[org].add(row['Venue_Year'])

edges = []
for org, venues in organizer_to_venues.items():
    for v1, v2 in itertools.combinations(sorted(venues), 2):
        edges.append((v1, v2, org))

edges_df = pd.DataFrame(edges, columns=['Source', 'Target', 'Organizer'])
edges_df['Weight'] = edges_df.groupby(['Source', 'Target'])['Organizer'].transform('count')
edges_df = edges_df.drop_duplicates(subset=['Source', 'Target']).sort_values(by='Weight', ascending=False)

# --- STEP 3: Create nodes with metadata and coordinates ---
title_agg = df.groupby('Venue_Year')['Title'].apply(
    lambda x: ' | '.join(sorted(set(str(i) for i in x if pd.notna(i))))
).reset_index()

organizer_agg = df.groupby('Venue_Year')['Organizer'].apply(
    lambda x: ' | '.join(sorted(set(str(i) for i in x if pd.notna(i))))
).reset_index()

meta_df = df[['Venue_Year', 'Venue', 'City', 'State', 'Country', 'Year Started',
              'Start_Date', 'End_Date', 'Event Duration (Days)', 'Latitude', 'Longitude']].drop_duplicates(subset='Venue_Year')

nodes_df = meta_df.merge(title_agg, on='Venue_Year').merge(organizer_agg, on='Venue_Year')

nodes_df = nodes_df.rename(columns={
    'Venue_Year': 'Id',
    'Venue': 'Venue_Name',
    'Year Started': 'Year',
    'Start_Date': 'Start Date',
    'End_Date': 'End Date',
    'Organizer': 'Organizers'
})

# --- STEP 4: Add Degree and Interval ---
G = nx.Graph()
G.add_edges_from([(src, tgt) for src, tgt, _ in edges])
nodes_df['Degree'] = nodes_df['Id'].map(dict(G.degree()))

def get_interval(year):
    if 2005 <= year <= 2009:
        return '2005 to 2009'
    elif 2010 <= year <= 2014:
        return '2010 to 2014'
    elif 2015 <= year <= 2019:
        return '2015 to 2019'
    elif 2020 <= year <= 2024:
        return '2020 to 2024'
    elif year == 2025:
        return '2025'
    else:
        return 'Other'

nodes_df['Interval'] = nodes_df['Year'].apply(get_interval)

# --- STEP 5: Remove special characters ---
def remove_accents(input_str):
    if isinstance(input_str, str):
        return unicodedata.normalize('NFKD', input_str).encode('ascii', 'ignore').decode('utf-8')
    return input_str

text_cols = ['Venue_Name', 'City', 'State', 'Country', 'Title', 'Organizers']
for col in text_cols:
    nodes_df[col] = nodes_df[col].apply(remove_accents)

edges_df['Organizer'] = edges_df['Organizer'].apply(remove_accents)

# --- STEP 6: Apply Jitter to overlapping nodes ---
jitter_strength = 0.005  # ~500m variation
np.random.seed(42)
nodes_df['lat'] = nodes_df['Latitude'] + (np.random.randn(len(nodes_df)) * jitter_strength)
nodes_df['lon'] = nodes_df['Longitude'] + (np.random.randn(len(nodes_df)) * jitter_strength)

# --- STEP 7: Merge lat/lon into edges and filter ---
geo_info = nodes_df[['Id', 'lat', 'lon']]
edges_geo = edges_df.merge(geo_info, left_on='Source', right_on='Id', how='left') \
                    .rename(columns={'lat': 'src_lat', 'lon': 'src_lon'}).drop(columns=['Id'])

edges_geo = edges_geo.merge(geo_info, left_on='Target', right_on='Id', how='left') \
                    .rename(columns={'lat': 'tgt_lat', 'lon': 'tgt_lon'}).drop(columns=['Id'])

# Add Interval to filter by matching node periods
edges_geo = edges_geo.merge(nodes_df[['Id', 'Interval']], left_on='Source', right_on='Id', how='left') \
                     .rename(columns={'Interval': 'Source_Interval'}).drop(columns=['Id'])

edges_geo = edges_geo.merge(nodes_df[['Id', 'Interval']], left_on='Target', right_on='Id', how='left') \
                     .rename(columns={'Interval': 'Target_Interval'}).drop(columns=['Id'])

edges_geo = edges_geo[edges_geo['Source_Interval'] == edges_geo['Target_Interval']]
edges_geo['Interval'] = edges_geo['Source_Interval']

# --- STEP 8: Final outputs ---
kepler_nodes = nodes_df.rename(columns={'Venue_Name': 'Venue'})
kepler_edges = edges_geo[['Source', 'Target', 'Organizer', 'Weight', 'Interval',
                          'src_lat', 'src_lon', 'tgt_lat', 'tgt_lon']]

kepler_nodes = kepler_nodes.fillna("")
kepler_edges = kepler_edges.fillna("")

In [4]:
kepler_nodes['Year (Time)'] = pd.to_datetime(kepler_nodes['Year'].astype(str), format='%Y') \
                                   .dt.strftime('%Y-%m-%d 00:00:00')

In [5]:
kepler_nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     442 non-null    object 
 1   Venue                  442 non-null    object 
 2   City                   442 non-null    object 
 3   State                  442 non-null    object 
 4   Country                442 non-null    object 
 5   Year                   442 non-null    int64  
 6   Start Date             442 non-null    object 
 7   End Date               442 non-null    object 
 8   Event Duration (Days)  442 non-null    int64  
 9   Latitude               442 non-null    float64
 10  Longitude              442 non-null    float64
 11  Title                  442 non-null    object 
 12  Organizers             442 non-null    object 
 13  Degree                 442 non-null    object 
 14  Interval               442 non-null    object 
 15  lat   

In [6]:
kepler_nodes.head()

,Id,Venue,City,State,Country,Year,Start Date,End Date,Event Duration (Days),Latitude,Longitude,Title,Organizers,Degree,Interval,lat,lon,Year (Time)
0,101st Annual Meeting of the Association of Ame...,101st Annual Meeting of the Association of Ame...,Denver,Colorado,United States,2005,2005-04-05,2005-04-09,5,39.739236,-104.984862,101st Annual Meeting of the Association of Ame...,"Deborah MacPherson, Katy Borner",69.0,2005 to 2009,39.741720,-104.995199,2005-01-01 00:00:00
1,Lorentz Workshop_2005,Lorentz Workshop,Leiden,,Netherlands,2005,2005-04-07,2005-04-11,5,52.159475,4.490884,Simulating the Social Processes of Science,Andrea Scharnhorst,15.0,2005 to 2009,52.158783,4.490439,2005-01-01 00:00:00
2,"SLIS, Indiana University_2005","SLIS, Indiana University",Bloomington,Indiana,United States,2005,2005-04-25,2005-04-25,1,39.167040,-86.534288,Networks and Complex Systems Talk Series,Peter A. Hook,2.0,2005 to 2009,39.170278,-86.540810,2005-01-01 00:00:00
3,Abdus Salam International Centre for Theoretic...,Abdus Salam International Centre for Theoretic...,Trieste,,Italy,2005,2005-05-15,2005-05-27,13,45.649648,13.777278,School and Workshop on Structure and Function ...,Katy Borner,63.0,2005 to 2009,45.657264,13.780626,2005-01-01 00:00:00
4,University of Illinois_2005,University of Illinois,Urbana-Champaign,Illinois,United States,2005,2005-05-17,2005-05-20,4,40.076154,-88.223313,Understanding Complex Systems 2004 Symposium,Peter A. Hook,2.0,2005 to 2009,40.074984,-88.221480,2005-01-01 00:00:00


In [7]:
kepler_edges.info()

<class 'pandas.core.frame.DataFrame'>
Index: 859 entries, 0 to 2309
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Source     859 non-null    object 
 1   Target     859 non-null    object 
 2   Organizer  859 non-null    object 
 3   Weight     859 non-null    int64  
 4   Interval   859 non-null    object 
 5   src_lat    859 non-null    float64
 6   src_lon    859 non-null    float64
 7   tgt_lat    859 non-null    float64
 8   tgt_lon    859 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 67.1+ KB


In [8]:
kepler_edges.head()

,Source,Target,Organizer,Weight,Interval,src_lat,src_lon,tgt_lat,tgt_lon
0,National Academy of Science Workshop_2010,Workshop_2010,Katy Borner,2,2010 to 2014,38.897647,-77.036185,38.896522,-77.038931
1,Monroe County Public Library_2007,New York Hall of Science_2006,Katy Borner,2,2005 to 2009,39.168758,-86.534328,40.713845,-73.828875
2,National Academy of Science Workshop_2010,National Academy of Sciences_2013,Katy Borner,1,2010 to 2014,38.897647,-77.036185,38.891173,-77.035552
7,Monroe County Public Library_2007,i-Light Symposium 2005_2005,Katy Borner,1,2005 to 2009,39.168758,-86.534328,39.765522,-86.163526
8,National Academy of Science Workshop_2010,National Academy of Sciences_2011,Katy Borner,1,2010 to 2014,38.897647,-77.036185,38.875609,-77.093262


In [9]:
kepler_edges['Source_Year'] = kepler_edges['Source'].str.extract(r'_(\d{4})$').astype(int)
kepler_edges['Target_Year'] = kepler_edges['Target'].str.extract(r'_(\d{4})$').astype(int)
kepler_edges['Edge_Year'] = kepler_edges[['Source_Year', 'Target_Year']].min(axis=1)
kepler_edges['Edge_Year (Time)'] = pd.to_datetime(kepler_edges['Edge_Year'].astype(str), format='%Y') \
                                        .dt.strftime('%Y-%m-%d 00:00:00')

In [10]:
kepler_edges.head()

,Source,Target,Organizer,Weight,Interval,src_lat,src_lon,tgt_lat,tgt_lon,Source_Year,Target_Year,Edge_Year,Edge_Year (Time)
0,National Academy of Science Workshop_2010,Workshop_2010,Katy Borner,2,2010 to 2014,38.897647,-77.036185,38.896522,-77.038931,2010,2010,2010,2010-01-01 00:00:00
1,Monroe County Public Library_2007,New York Hall of Science_2006,Katy Borner,2,2005 to 2009,39.168758,-86.534328,40.713845,-73.828875,2007,2006,2006,2006-01-01 00:00:00
2,National Academy of Science Workshop_2010,National Academy of Sciences_2013,Katy Borner,1,2010 to 2014,38.897647,-77.036185,38.891173,-77.035552,2010,2013,2010,2010-01-01 00:00:00
7,Monroe County Public Library_2007,i-Light Symposium 2005_2005,Katy Borner,1,2005 to 2009,39.168758,-86.534328,39.765522,-86.163526,2007,2005,2005,2005-01-01 00:00:00
8,National Academy of Science Workshop_2010,National Academy of Sciences_2011,Katy Borner,1,2010 to 2014,38.897647,-77.036185,38.875609,-77.093262,2010,2011,2010,2010-01-01 00:00:00


In [11]:
kepler_nodes.to_excel("kepler_nodes.xlsx", index=False)
kepler_edges.to_excel("kepler_edges.xlsx", index=False)